In [3]:
%matplotlib notebook

from matplotlib import rcParams
# Set the font of the plots:
rcParams['font.family'] = 'Helvetica'
rcParams['font.sans-serif'] = ['Helvetica']
import matplotlib.pyplot as plt
import networkx as nx
import toverboom
import toverboom.lineageGraph
import toverboom.optimizeLayout
import toverboom.preprocessing
import pandas as pd

# Development:
import importlib
importlib.reload(toverboom)
importlib.reload(toverboom.lineageGraph)
importlib.reload(toverboom.preprocessing)
importlib.reload(toverboom.optimizeLayout)

ModuleNotFoundError: No module named 'toverboom'

In [ ]:
# Load the graph file into a networkx graph
graph = nx.read_graphml(f'../data/APKS1_CNV_tree.graphml')
graph = toverboom.preprocessing.convertFromStringKeys(graph)

In [ ]:
# Instantiate the lineage graph object
lg = toverboom.lineageGraph.LineageGraph(graph)

# Create a figure for our plot:
fig, ax = plt.subplots()

# Find the best layout
toverboom.optimizeLayout.optimize_layout(lg,visualize_progress_ax=ax, visualize_progress_fig=fig)

In [ ]:
# Plot the polygons of the tree
fig, ax = plt.subplots()

# wavyness controls how wavy the segments of the tree are 
wavyness=0.4
# xDistance controls the stretch in the x direction 
lg.xDistance=10
lg.verticalSpacing = 0.1

lg.plotEdges(ax, bezier=True,wavyness=wavyness,stepCount=30,plotArgs={'linewidth':1}, offsetCentroid=True)
lg.plotPatches(ax=ax,wavyness=wavyness)

# Remove plot spines:
toverboom.lineageGraph.despine(ax)
# Scale labels to plot size:
toverboom.lineageGraph.format_x_axis_labels(ax)

# Add labels to the clones:
lg.annotateNodes(ax,plotArgs={'size':8})

fig.canvas.draw()

## Plot single cells on top of the tree

First construct a matrix where the single cells are described
Required columns are:
```
tp : timepoint where the cell was measured
cluster: to what cluster/clone does the cell belong
```
Optional columns are:
```
size : plotted size of the cell
marker : marker 
color  : color of the marker
edge_width : thickness of marker edge
edge_color : color of the edge of the marker
label : label of the cell
x : x offset of the cell
y : y offset of the cell
z-order : rendering position (z-index) of the cell
```

In [ ]:
# Load example data:
replicate = 'APKS1'
cellBarcodes = pd.read_pickle('../data/cellToBarcode.pickle').loc[replicate]
cellCnv = pd.read_pickle('../data/singleCellCNVClustering.pickle.gz').loc[replicate]
cellData = cellCnv

#### Contruct a dataframe with cell data

# Assign colors:
cellData['color'] = [{
    3:'r', 16:'r', 20:'r' ,
    5:'c',  21:'c',
    2:'orange'}.get(cluster,'grey') for cluster in cellData['cluster'] ]

# Assign passage:
cellData['tp'] = [passage for passage, plate, cell in list(cellData.index)]

# Assign sizes
cellData['size'] = [100 if cluster==2 else 60 for cluster in cellData['cluster'] ]

# Assign markers
cellData['marker'] = [{3:'s', 5:'s', 1:'*'}.get(cluster,'.') for cluster in cellData['cluster'] ]

# Assign edge width
cellData['edge_width'] = [1.5 if cluster==2 else 1 for cluster in cellData['cluster'] ]

# Assign order of plotting
cellData['z-order'] = [1 if cluster==5 else 0 for cluster in cellData['cluster'] ]

cellData.head()

In [ ]:
lg.plotSingleCells(cellData,plotPatches=True,enableShadow=True)
ax = plt.gca()
# Add labels to the clones:
lg.annotateNodes(ax,plotArgs={'size':9}, 
                 # Use the nodesToAnnotate argument to select which nodes to annotate
                 nodesToAnnotate=[
                     (cluster,tp)
                      for cluster,tp in lg.graph if cluster in [1,4,3,5,2,8]],
                 x_offset = 8 # How much to move the labels to the right
                )

# Add vertical lines to indicate sampled timepoints
lg.plot_vertical_lines(ax, cellData['tp'].unique(), c='red')
ax.set_xlabel('Time [weeks]')


In [ ]:
# Defining X/Y offset for cells
cellData

## Color the tree patches

First construct a dataframe where the patches are described
The patches are EDGES of your graph, they are defined by 4 values:
starting clone, starting timepoint
ending clone, ending timepoint

```
fromTp : timepoint 
fromCluster: cluster
toTp : timepoint 
toCluster: cluster
```

All valid matplotlib colors can be used such as: 
``` 'green','r',(1,0,0), (1,0,0,0.5)``` 

To obtain which edges you can color run this command:
``` list(lg.graph.edges())```

In [ ]:
patchData =  pd.DataFrame(
    {
        ((2, 10), (3, 22)):{'color':'red', #patch fill is red
                            'edgecolor':'k', # edge is black
                            'linewidth':2, # edge is 2 pixels
                            'linestyle':'-', # Draw a filled edge
                            'zorder':50 # Put this patch in front
                           },
        ((2, 10), (5, 22)):{'color':(0,0.5,0),'zorder':1  },
        ((2, 10), (2, 22)):{'color':(0,0.5,0),'edgecolor':'white','zorder':1 },
        ((0, 0), (2, 10)):{'color':(0,0.5,0),'edgecolor':'white','zorder':1 }
    }).T
patchData['linewidth'].fillna(0, inplace=True)
patchData

In [ ]:
fig,ax = plt.subplots()
# Make plot scale the right way by plotting invisible edges:
lg.plotEdges(ax,plotArgs={'linewidth':0})

#Plot the patches using the matrix we just made
lg.plotPatches(ax,
               facecolor=(1,1,1), # Default face color, 
                               #None to not plot patches not 
                               # described in patchData
               linestyle='--',
               linewidth=0.5,
               zorder=0,
               edgecolor='grey',
               patchData=patchData)

# Remove plot spines:
toverboom.lineageGraph.despine(ax)
# Scale labels to plot size:
toverboom.lineageGraph.format_x_axis_labels(ax)

fig.canvas.draw()